In [2]:
import os
import numpy as np
import pandas as pd
import pickle

## Getting momentum variables and current tournament points

In [2]:
df = pd.read_csv("../data/cleaned_data.csv")

# ADDING ACCURACY AND CONVERSION VARIABLES
# Accuracy is ratio of shots on target among shots
df["HAcc"] = df.apply(lambda x: x["HST"] / (x["HS"] + 1e-5), axis = 1)
df["AAcc"] = df.apply(lambda x: x["AST"] / (x["AS"] + 1e-5), axis = 1)

# Conversion is ratio of goals among shots
df["HConv"] = df.apply(lambda x: x["FTHG"] / (x["HS"] + 1e-5), axis = 1)
df["AConv"] = df.apply(lambda x: x["FTAG"] / (x["AS"] + 1e-5), axis = 1)

# Initializing points columns
df["HCurrentPoints"] = 0
df["ACurrentPoints"] = 0

metrics = ['FT{}G', '{}S', '{}ST', '{}Acc', '{}Conv']
refs = ['H', 'A']
momentum_var = [mom.format(ref) + '_5Avg' for mom, ref in [(m, r) for m in metrics for r in refs]]
for mom in momentum_var: df[mom] = 0


In [3]:
for idx in range(len(df)):
    
    row = df.iloc[idx].copy()
    season = row.Season
    date = row.Date
    home_team = row.HomeTeam
    away_team = row.AwayTeam
    # Last 5 games of Home team
    df_season_home = df[((df.HomeTeam == home_team) | (df.AwayTeam == home_team)) & (df.Date < date) & (df.Season == season)].sort_values(by='Date')[-5:]
    # Last 5 games of Away team
    df_season_away = df[((df.HomeTeam == away_team) | (df.AwayTeam == away_team)) & (df.Date < date) & (df.Season == season)].sort_values(by='Date')[-5:]

    momentum = {mom: [] for mom in momentum_var}

    # Getting Home last values
    if  len(df_season_home) == 0:
        for metric in metrics: momentum[metric.format('H') + '_5Avg'].append(np.nan)
        row.HCurrentPoints = 0
        
    else:
        for i, (_, row_home) in enumerate(df_season_home.iterrows()):
            if row_home.HomeTeam == home_team: ref = 'H'
            else: ref = 'A'
            for metric in metrics: momentum[metric.format('H') + '_5Avg'].append(row_home[metric.format(ref)])
            
            # Updating Home current points in the tournament
            if i == len(df_season_home)-1:
                old_points = row_home['{}CurrentPoints'.format(ref)]
                result = row_home.FTR
                if result == ref: new_points = old_points + 3
                elif result == 'D': new_points = old_points + 1
                else: new_points = old_points
                row.HCurrentPoints = new_points
            
    # Getting Away last values
    if  len(df_season_away) == 0:
        for metric in metrics: momentum[metric.format('A') + '_5Avg'].append(np.nan)
        row.ACurrentPoints = 0
        
    else:
        for i, (_, row_away) in enumerate(df_season_away.iterrows()):
            if row_away.HomeTeam == away_team: ref = 'H'
            else: ref = 'A'
            for metric in metrics: momentum[metric.format('A') + '_5Avg'].append(row_away[metric.format(ref)])    
            
            # Updating Away current points in the tournament
            if i == len(df_season_away)-1:
                old_points = row_away['{}CurrentPoints'.format(ref)]
                result = row_away.FTR
                if result == ref: new_points = old_points + 3
                elif result == 'D': new_points = old_points + 1
                else: new_points = old_points
                row.ACurrentPoints = new_points
    
    # Adding momentum columns to row
    for metric in metrics:
        row[metric.format('H') + '_5Avg'] = np.array(momentum[metric.format('H') + '_5Avg']).mean()
        row[metric.format('A') + '_5Avg'] = np.array(momentum[metric.format('A') + '_5Avg']).mean()
        
    df.loc[idx, :] = row


## Adding optimal implicit ratings from lectures' analysis

In [4]:
ratings = pd.read_csv('../data/optimal_ratings.csv')
def get_season_more(s):
    return s[-4:] + '-' + str(int(s[-4:]) + 1)
ratings['season+'] = ratings.season.apply(get_season_more)

merge_home = pd.merge(df, ratings, how = 'left', left_on = ['Season', 'HomeTeam'], right_on = ['season+', 'team'], left_index = True)
merge_away = pd.merge(merge_home, ratings, how = 'left', left_on = ['Season', 'AwayTeam'], right_on = ['season+', 'team'], left_index = True)

df_ratings = merge_away.drop(columns = ['season_x', 'season+_x', 'team_x', 'season_y', 'season+_y', 'team_y'])
df_ratings = df_ratings.rename(columns = {'rating_x': 'HRating', 'rating_y': 'ARating'})

## Getting data exploitable for models

In [5]:
df_final = df_ratings.copy()

# Removing 2012-2013 season because no data on optimal ratings
df_final = df_final[df_final.Season > '2012-2013']

# Getting number of previous games in the season for each team
def get_nb_previous_games(row):
    
    season = row.Season
    date = row.Date
    home_team = row.HomeTeam
    away_team = row.AwayTeam

    row['prev_home'] = len(df_final[((df_final.HomeTeam == home_team) | (df_final.AwayTeam == home_team)) & (df_final.Date < date) & (df_final.Season == season)])
    row['prev_away'] = len(df_final[((df_final.HomeTeam == away_team) | (df_final.AwayTeam == away_team)) & (df_final.Date < date) & (df_final.Season == season)])
    
    return row

df_final = df_final.apply(get_nb_previous_games, axis = 1)

# Getting rid of first 2 games of every team in each season
df_filtered = df_final[(df_final.prev_home >= 2) & (df_final.prev_away >= 2)].drop(columns = ['prev_home', 'prev_away'])

# Filling remaining NA ratings with 0 (this concerns all games of the 3 new teams of every season)
df_filtered = df_filtered.fillna(0)

# Normalizing team values columns
df_norm = df_filtered.copy()
df_norm.AValue = df_norm.groupby('Season').AValue.transform(lambda x: (x - x.min()) / (x.max()- x.min()))
df_norm.HValue = df_norm.groupby('Season').HValue.transform(lambda x: (x - x.min()) / (x.max()- x.min()))

# Getting differences of metrics
metrics = ['{}Value', '{}CurrentPoints', '{}Rating',
           'FT{}G_5Avg', '{}S_5Avg', '{}ST_5Avg', '{}Acc_5Avg', '{}Conv_5Avg']
for metric in metrics:
    df_norm[metric.format('') + '_Diff'] = df_norm[metric.format('H')] - df_norm[metric.format('A')]
    
    
df_norm.to_csv('../data/final_data.csv', index = False)